In [ ]:
from functools import partial

import numpy as np
import pandas as pd
from python.hydro import DownhillSimplex, make_simplex, calibration, nse, nse_min

# there is a pure-Python implementation of GR4J (python.hydro)
# pydro is a Cython implementation and is much faster
# from python.hydro import GR4J  # slow
from pydro.pydro import GR4J  # fast

The data comes from [airGR](), and was saved to CSV using these commands in an R console:
```R
install.packages("airGR")
library(airGR)
data(L0123001)
write.csv(BasinObs, "airGR_L012301.csv")
```
It was then included in the bundle for this web site, and can be accessed in the `data` directory.

We load it in [pandas](https://pandas.pydata.org).

In [ ]:
df = pd.read_csv("data/airGR_L012301.csv")
peq = df.rename(columns={"DatesR": "t", "P": "p", "E": "e", "Qmm": "q"})
peq = peq.set_index(pd.to_datetime(peq.t))[["p", "e", "q"]]

Now we calibrate the [GR4J](https://webgr.inrae.fr/en/models/daily-hydrological-model-gr4j) hydrological model using precipitation and PET as inputs, and observed discharge as output.

In [ ]:
x_range = [[0., np.inf], [-np.inf, np.inf], [0., np.inf], [0., np.inf]]
x_init = [100, 0, 100, 1]
warmup_period = 365
x, *_ = DownhillSimplex(
    partial(
        calibration,
        in_obs=[peq.p.values, peq.e.values],
        out_obs=[peq.q.values],
        warmup_period=warmup_period,
        crit_func=nse_min,
        model=GR4J,
        x_range=x_range
    ),
    make_simplex(x_init)).run()
q_mod = GR4J(x)
peq["q_sim"] = q_mod.run([peq.p.values, peq.e.values])[0]
print(f"{x=}")
print(f"NSE={nse(peq.q, peq.q_sim)}")